In [9]:
import numpy as np

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR

import torchvision

from tqdm.notebook import tqdm

import train as training

In [10]:
dire = "./Data/Processed" # directory of dataset
# loading data loader
trainloader, testloader = training.preprocess_data(directory=dire, batch_size=64, test_size=0.3, rand_num=40, worker=2)

{0: 100, 1: 171, 2: 76, 3: 78}


In [11]:
# load pretrained model
vgg16 = torchvision.models.vgg16(pretrained=True)
googlenet = torchvision.models.googlenet(pretrained=True)
efficientnet_b7 = torchvision.models.efficientnet_b7(pretrained=True)
resnet50 = torchvision.models.resnet50(pretrained=True)

In [12]:
class ClassifierNet(nn.Module):
    def __init__(self, pretrained, num_classes):
        super().__init__()
        # Pretrained
        self.network = pretrained
        # Replace last layer
        self.network.fc = nn.Sequential(nn.Linear(self.network.fc.in_features, 512), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.25),
                                         nn.Linear(512, 128), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.50), 
                                         nn.Linear(128, num_classes),
                                         nn.Softmax()
                                        )
    def forward(self, x):
        out = self.network(x)
        return out


In [13]:
# define model to replace here
model_details = {
    'model': resnet50,
    'name': 'resnet50'

#     'model': efficientnet_b7,
#     'name': 'efficientnet_b7'

#     'model': googlenet,
#     'name': 'googlenet'

#     'model': vgg16,
#     'name': 'vgg16' 
}

In [14]:
# define params here
num_classes = 4
model_name = model_details['name']
net = ClassifierNet(model_details['model'], num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = ExponentialLR(optimizer, gamma=0.9)

criterion = nn.CrossEntropyLoss()
epochs = 50

training.run(model_name, net, optimizer, scheduler, criterion, epochs, num_classes, trainloader, testloader)

Training on cuda


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/50], Step [5/5], Accuracy: 28.523, Train Loss: 1.3821
Epoch 1 Learning Rate: 0.009000000000000001
-----------------------------------------------
Epoch [2/50], Step [5/5], Accuracy: 36.577, Train Loss: 1.3855
Epoch 2 Learning Rate: 0.008100000000000001
-----------------------------------------------
Epoch [3/50], Step [5/5], Accuracy: 39.933, Train Loss: 1.3600
Epoch 3 Learning Rate: 0.007290000000000001
-----------------------------------------------
Epoch [4/50], Step [5/5], Accuracy: 40.268, Train Loss: 1.3699
Epoch 4 Learning Rate: 0.006561000000000002
-----------------------------------------------
Epoch [5/50], Step [5/5], Accuracy: 40.268, Train Loss: 1.3438

===================Validating===================
Epoch: 5 | Test Loss: 1.350 | Accuracy: 40.157
Class accuracy: [0.0, 1.0, 0.0, 0.0]

Epoch 5 Learning Rate: 0.005904900000000002
-----------------------------------------------
Epoch [6/50], Step [5/5], Accuracy: 40.268, Train Loss: 1.3388
Epoch 6 Learning Rate: 0.00